# WaveNet
![wavenet](../images/wavenet.png)

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

/home/barbara/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
words = open("../data/names.txt", 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
# Vocabulary
chars = [c for c in '.abcdefghijklmnopqrstuvwxyz']

# Convertion between characters and integers
itos = {i: c for i, c in enumerate(chars)}
stoi = {c: i for i, c in enumerate(chars)}

In [4]:
vocab_size = len(chars)
block_size = 8
n_hidden = 128
n_emb = 24

In [5]:

def build_dataset(names):
    X, Y = [], []

    for name in names:
        c1 = [0] * block_size
        for ch in name + '.':
            X.append(c1)
            Y.append(stoi[ch])
            c1 = c1[1:] + [stoi[ch]]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)

    return X, Y

In [6]:
class Linear:

    def __init__(self, fan_in, fan_out, bias=True):
        self.weights = torch.randn((fan_in, fan_out)) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weights
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weights] + ([] if self.bias is None else [self.bias])

Great video about normalizations: https://www.youtube.com/watch?v=sxEqtjLC0aM

In [7]:

class BatchNorm:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True # True for training, False for evaluation

        # parameters
        self.gamma = torch.ones(dim)  # bn weight
        self.beta = torch.zeros(dim)  # bn bias

        # buffers (trained with a momentum update / not part of the backpropagation update)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)


    def __call__(self, x):
        if self.training:
            if x.ndim == 2:
                dim = 0
            elif x.ndim == 3:
                dim = (0,1)
            
            x_mean = x.mean(dim, keepdim=True)
            x_var = x.var(dim, keepdim=True)
        else:
            x_mean = self.running_mean
            x_var = self.running_var
        x_hidden_act = (x - x_mean) / torch.sqrt(x_var + self.eps) 
        self.out = self.gamma * x_hidden_act + self.beta

        # update running buffers
        if self.training:
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * x_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * x_var

        return self.out

    def parameters(self):
        return [self.gamma, self.beta]
            


In [8]:
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out

    def parameters(self):
        return []

In [9]:
class Embedding:
    def __init__(self, num_embeddings, embedding_dim):
        self.weights = torch.randn(num_embeddings, embedding_dim)
    
    def __call__(self, IX):
        self.out = self.weights[IX]
        return self.out 

    def parameters(self):
        return [self.weights]
    

In [10]:
class FlattenConsecutive:
    def __init__(self, n):
        self.n = n
    
    def __call__(self,x):
        B, T, C = x.shape
        x = x.view(B, T//self.n, C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
            
        self.out = x
        return self.out

    def parameters(self):
        return []

In [11]:
class Sequential:
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        self.out = x
        return self.out
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [12]:
torch.manual_seed(42)

model = Sequential([
    Embedding(vocab_size, n_emb),
    FlattenConsecutive(2), Linear(n_emb * 2, n_hidden, bias=False),  BatchNorm(n_hidden), Tanh(),
    FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False),  BatchNorm(n_hidden), Tanh(),
    FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False),  BatchNorm(n_hidden), Tanh(),
    Linear( n_hidden, vocab_size)
])

with torch.no_grad():
    # last layer: make less confident
    model.layers[-1].weights *= 0.1
    # all other layers: apply gain
    # for layer in model.layers:
    #     if isinstance(layer, Linear):
    #         layer.weights *= 5/3

parameters = model.parameters()
print("parameters:", sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

parameters: 76579


In [13]:
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [14]:
max_steps = 200000
batch_size = 32
losses = []
b = 0
for layer in model.layers:
    layer.training = True

for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size, ))
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward passs
    logits = model(Xb)
    loss = F.cross_entropy(logits, Yb)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 150000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    
    
    if(i % (max_steps//20) == 0):
        b+=1
        print(f"{i/max_steps*100:3.3}% complete - loss: {loss.item()}")
    losses.append(loss.log10().item())
    
    

0.0% complete - loss: 3.3167154788970947
5.0% complete - loss: 2.0576305389404297
10.0% complete - loss: 2.0723214149475098
15.0% complete - loss: 2.513370990753174
20.0% complete - loss: 2.147610902786255


In [ ]:
plt.plot(torch.tensor(losses).view(-1,100).mean(1))

## Evaluate the model

In [ ]:
@torch.no_grad()
def split_loss(split):
    for layer in model.layers:
        layer.training = False
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte)
    }[split]
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

# Sample the model

In [ ]:
num_samples = 20

for layer in model.layers:
    layer.training = False
    
for _ in range(num_samples):
    out = []
    context = [0] * block_size
    while True:
        logits = model(torch.tensor([context]))
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join([itos[i] for i in out[:-1]]))